# File Upload

In [ ]:
from google.colab import files


# csv file
uploaded = files.upload()
for f in uploaded.keys():
    print('User iploaded file "{name}" with length {length} bytes\n'.format(name = f, length = len(uploaded[f])))

# Task 

## Model: $y = f(x)$
## Approach: Neural Network

## 1. Read the .csv file and store the data into the dataframe 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


plt.figure(figsize=(10, 7))
plt.title("Real Data")

data = pd.read_csv("./hw3_data.csv").sort_values(by='x', axis=0)
plt.plot(data['x'], data['y'], 'bo', label="Data Set")
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

## 2. Define the perceptron used in the multi-layer perceptron (MLP)

### 2.1 Define Activation Functions

In [ ]:
import numpy as np


class Sigmoid:
    def forward(self, x):
        return 1.0 / (1.0 + np.exp(-x))

    def derivate(self, x):
        return (1 - self.forward(x)) * self.forward(x)


class Relu:
    def forward(self, x):
        return np.where(x > 0, x, 0.0)

    def derivate(self, x):
        return np.where(x > 0, 1.0, 0.0)


class Tanh:
    def forward(self, x):
        return np.tanh(x)

    def derivate(self, x):
        return 1 - np.tanh(x) ** 2


class Identity:
    def forward(self, x):
        return x

    def derivate(self, x):
        return 1

### 2.2 Define Initializer

In [ ]:
import numpy as np


class Rand:
    def initialize(self, layer):
        w = np.random.rand(layer.input_dim, layer.output_dim)
        b = np.random.randd(1, layer.output_dim)
        dw = np.zeros([layer.input_dim, layer.output_dim])
        db = np.zeros([1, layer.output_dim])
        return w, b, dw, db

### 2.3 Define Layer

In [ ]:
import numpy as np


class Dense:
    def __init__(self, units, activation, input_dim, initializer=Rand):
        self.w = None
        self.b = None
        self.z = None
        self.a = None
        self.dw = None
        self.db = None
        self.delta = None

        self.output_dim = units
        self.input_dim = input_dim
        self.activation = activation
        self.initializer = initializer


    def reset_layer(self):
        w, b, dw, db = self.initializer.initialize(self)
        self.w = w
        self.b = b
        self.dw = dw
        self.db = db


    def forward(self, x, update):
        z = np.matmul(x, self.w) + self.b
        a = self.activation.forward(z)
        if update:
            self.z = z
            self.a = a
        return a

    
    def update_delta(self, next_layer):
        delta = np.matmul(next_layer.delta, next_layer.w.T) * self.activation.derivate(self.z)
        self.delta = delta


    def update_gradient(self, a_in):
        delta_out = self.delta
        self.db = delta_out.sum(axis=0).reshape([1, -1])
        self.dw = np.matmul(a_in.T, delta_out)

### 2.4 Define Loss

In [ ]:
import numpy as np


class MSE:
    def forward(self, actual, prediction):
        return 0.5 * ((prediction - actual) ** 2)

    def derivate(self, actual, prediction):
        return prediction - actual

### 2.8 Define Optimizer

In [ ]:
class GradientDescent:
    def __init__(self, learning_rate=0.001):
        self.learning_rate = learning_rate

    
    def initialize_weights(self, layers):
        return layers

    
    def update_weights(self,layers):
        for i in range(len(layers)):
            layers[i].w = layers[i].w - self.learning_rate * layers[i].dw
            layers[i].b = layers[i].b - self.learning_rate * layers[i].db
        return layers

### 2.5 Define Model

In [ ]:
import numpy as np


class MLP:
    def __init__(self):
        self.layers = []
        self.n_layers = 0
        self.trainer = None
        self.train_log = None


    def add(self, layer):
        layer.input_dim = self.layers[-1].output_dim
        layer.reset_layer()
        self.layers.append(layer)
        self.n_layers += 1


    def predict(self, x):
        p = self.forward_prop(x, update=False)
        return p


    def train(self, loss, x, optimizer=GradientDescent):
        self.trainer = ModelTrain()
        self.trainer.train(self, loss, x, optimizer)


    def forward_prop(self, x, update=True):
        a = x
        for layer in self.layers:
            a = layer.forward(a, update=update)
        return a


    def back_prop(self, x, y, loss):
        self.update_deltas(loss, y)
        self.update_gradients(x)


    def update_deltas(self, loss, y):
        for i, layer in enumerate(reversed(self.layers)):
            if i == 0:
                delta = loss.derivate(y, layer.a) * layer.activation.derivate(layer.z)
                layer.delta - delta
            else:
                layer_next = self.layers[-i]
                layer.update_delta(layer_next)

    
    def update_gradients(self, x):
        for i, layer in enumerate(self.layers):
            if i == 0:
                a_in = x
            else:
                prev_layer = self.layers[i - 1]
                a_in = prev_layer.a
            layer.update_gradient(a_in)

### 2.7 Define Batcher

In [ ]:
import numpy as np


class Batcher:
    def __init__(self, x, batch_size):
        self.x = x
        self.batch_size = batch_size
        self.shuffle_on_reset = False

        if type(x) == list:
            self.data_size = data[0].shape[0]
        else:
            self.data_size = data.shape[0]
        
        self.n_batches = int(np.cceil(self.data_size / self.batch_size))
        self.idx = np.arrange(0, self.data_size, dtype=int)
        self.current = 0

    
    def shuffle(self):
        np.random.shuffle(self.idx)


    def reset(self):
        if self.shuffle_on_reset:
            self.shuffle()
        self.current = 0


    def next(self):
        batch = []
        i_select = self.idx[(self.current * self.batch_size) : ((self.current + 1) * self.batch_size)]

        for data in self.x:
            batch.append(data[i_select])
        
        if self.current < (self.n_batches - 1):
            self.current = self.current + 1
        else:
            self.reset()

        return batch

### 2.6 Define Trainer

In [ ]:
import numpy as np
import pandas as pd


default_params = {
    'n_epoch': 10,
    'print_rate': 5,
    'batch_size': 128,
    'learning_rate': 0.001
}


class ModelTrain:
    def __init__(self):
        self.batcher = None
        self.optimizer = None
        self.params = default_params
        

    def train(self, model, loss, x, optimizer=GradientDescent):
        self.optimizer = optimizer
        model.layers = self.optimizer.initialize_parapmeters(model.layers)

        if self.batcher is None:
            self.batcher = Batcher(x, self.params['batch_size'])

        epoch = 1
        train_loss = []
        model.train_log = []
        while epoch <= self.params['n_epoch']:
            self.batcher.reset()

            for batch_i in range(self.batcher.n_batches):
                batch = self.batcher.next()
                x_batch = batch[0]
                y_batch = batch[1]

                self.train_step(model, x_batch, y_batch, loss)
                loss_i = self.compute_loss(model.layerrrs[-1].a, y_batch, loss)
                model.train_log.append(np.array([epoch, batch_i, loss_i]))

            epoch += 1

        model.train_log = np.vstack(model.train_log)
        model.train_log = pd.DataFrame(model.train_log, colums=['epoch', 'iter', 'loss'])


    def train_step(self, model, x, y, loss):
        _ = model.forward_prop(x)
        model.back_prop(x, y, loss)
        model.layers = self.optimizer.update_weights(model.layers)


    def compute_loss(actual, prediction, loss):
        current_loss = loss.forward(actual, prediction)
        return current_loss.mean()

## 3. Main

In [ ]:
model = MLP
model.add(Dense(units=8, activation=Relu, input_dim=1))
model.add(Dense(units=4, activation=Relu))
model.add(Dense(units=4, activation=Relu))
model.add(Dense(units=1, activation=Identity))

loss = MSE
print(model.layers)
# model.train(loss, [data['x'], data['y']])

# p = model.predict(data['x'])
# performance = loss.forward(p, data['y'])
# print(f"Prediction loss: {performance.mean()}")